In [1]:
import asyncio
from typing import Optional
from contextlib import AsyncExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [2]:
import ollama 
from ollama import Client

class MCPClient:
    def __init__(self):
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.anthropic = Client()

    async def connect_to_server(self, server_script_path: str):
        """Connect to an MCP server
        Args:
        server_script_path: Path to the server script (.py or .js)
         """
        is_python = server_script_path.endswith('.py')
        if not is_python:
            raise ValueError("Server script must be a .py file")
        
        server_params = StdioServerParameters(
        command="python",
        args=[server_script_path],
        env=None)

        stdio_transport = await self.exit_stack.enter_async_context(stdio_client(server_params))
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(ClientSession(self.stdio, self.write))

        await self.session.initialize()

        response = await self.session.list_tools()
        tools = response.tools
        print("\nConnected to server with tools:", [tool.name for tool in tools])

In [3]:
client = MCPClient()
path = "D:\\Coding Stuff\\GitHub\\Deeplearning.ai\\7. MCP Build Rich-Context AI Apps with Anthropic\\Weather_Server.py"
try:
    await client.connect_to_server(server_script_path=path)
finally:
    print("Done")

Done


NotImplementedError: 